# **Detección de Derrames de Petróleo Offshore**
Este proyecto fue desarrollado durante las últimas semanas de 3 meses de aprendizaje junto a Saturdays AI Quito. Este proyecto se diseñó utilizando Google Earth Engine, una plataforma basada en la nube que proporciona soporte para análisis geoespacional a escala planetaria. [Gorelick et al. (2017)](https://doi.org/10.1016/j.rse.2017.06.031)

**Integrantes:**
> Juan Carlos Cedeño, Diego Yanez, Pablo Quilachamin, María Belén Bedoya, Karla Uvidia Zambrano

**Instalar dependencias necesarias**

In [ ]:
!pip3 install earthengine-api --upgrade
!pip3 install mapbox
!pip3 install cmocean

**Importar dependencias**

In [35]:
# Procesamiento
import ee # Google Earth Engine, procesamiento geoespacial
import numpy as np #
import pandas as pd

# Visualización
import matplotlib.pyplot as plt # 
import cmocean.cm as cmo # Rampas de colores científicos

# Decodificación
import json
from io import BytesIO

# Maps
import folium
from mapbox import Uploader
from folium.plugins import MiniMap
from datetime import datetime, timedelta

**Autentificación de Google Earth Engine**

In [36]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## **AOI**
El Área de Interés (AOI) fue Kuwait, un país árabe del Golfo Pérsico. El 10 de agosto de 2017, se dió un evento de derrame de petróleo de aproximadamente 35.000 barriles frente a su costa sur.

In [37]:
# Kuwait, Golfo Pérsico. WGS84 Lat 28.51 Lon 48.56, 2017:08:[09-11] (S1)
lat, lon = [28.6, 48.6]
start, end = ['2017-08-09', '2017-08-11']
proj = 'EPSG:4326'
roi = ee.Geometry.Point([lon, lat], proj)
aoi = ee.Geometry.Rectangle(48.4023, 28.7623, 48.69, 28.48)

## **Adquirir datos desde la constelación de Copérnicus: Sentinel-1**
Google Earth Engine cuenta con toda la constelación de satélites desarrollados por la Agencia Espacial Europea (ESA) para el programa de Copernicus, y son de nuestro interés los satélites de Sentinel-1A y 1B.

In [38]:
# colección de Sentinel-1 acotada en espacio, tiempo, polarización y resolución
collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
              .filter(ee.Filter.eq('transmitterReceiverPolarisation', ['VV', 'VH']))
              .filter(ee.Filter.eq('instrumentMode', 'IW'))
              .filterDate(start , end)
              .filterBounds(roi)
              .filterMetadata('resolution_meters', 'equals', 10))

### **Fecha de adquisitión de la imagen satelital**

In [39]:
def acquisition_date(image_collection):
    """Extrae la fecha de adquisición de la imagen SAR usando la nomenclatura del producto (ID) de Sentinel-1"""

    # formatear el string a ISO 8601 (tiempo)
    def parse_time(t):
        return datetime.strptime(t, '%Y%m%dT%H%M%S')

    # almacenar identificador
    dict_id = collection.first().getInfo()['id']
    # lista de comprensión para extraer solo fechas del paso del satélite
    datetime_iso = [parse_time(i) for i in dict_id.split('_') if len(i) == 15]

    return datetime_iso[0]

print('Fecha de adquisición de la imagen SAR: {}'.format(
    acquisition_date(collection)))

Fecha de adquisición de la imagen SAR: 2017-08-10 02:47:14


## **Corrección del ángulo de incidencia**

Un efecto que puede restringir el análisis de las imágenes de SAR, principalmente en aplicaciones oceanográficas que asocian los procesos oceánicos y las características de la superficie del mar con objetivos oscuros (ej: derrames de petróleo), es la tendencia de disminución de la señal de retropropagación $\sigma°$ a medida que el ángulo de incidencia ($\theta$) incrementa. ([Alpers y Huang, 2011](https://doi.org/10.1109/TGRS.2010.2072930); [Singha, Vespe, y Trieschmann, 2013](https://doi.org/10.1016/j.marpolbul.2013.05.022))

El método utilizado para este trabajo se conoce como Square Cosine Correction (SCC), y asume que $\sigma°$ es una función del cuadrado del coseno de $\theta$.

In [40]:
def square_angle_correction(image):
    """Método de corrección de ángulo cuadrado"""
    return image.multiply(image.select('angle').multiply(np.pi/180).cos().pow(ee.Image(2)))

## **Subset**
Subset es una operación común en los Sistemas de Información Geográfica, utilizada para focalizar el área de estudio, su función es recortar los ráster usando una extensión definida.

In [41]:
def subset(image):
    """Método de corrección de ángulo cuadrado"""
    return image.clip(aoi)

## **La máscara offshore**
El objetivo es detectar los eventos **offshore**, es decir aquellos derrames de petróleo que se han dado costa fuera, por tanto es necesario retirar todo lo relacionado a la plataforma continental. 

Dentro del catálogo de imágenes satelitales de Google Earth Engine, podemos encontrar máscaras de agua, tales como: MOD44W V6, un producto que combina los datos de Shuttle Radar Topography Mision (SRTM) y Moderate Resolution Imaging Spectroradiometer (MODIS) de la NASA, para mapear la superficie de agua a nivel global con una resolución de 250 metros.


In [42]:
def ocean_mod44w_mask(image):
    """El MOD44W V6 Land/Water Mask es un producto derivado de MODIS basado en un clasificador de árbol de decisión
     y validado con el producto MOD44W V5."""
    # colección de MOD44W.006, 2015 última máscara disponible
    mod44w = ee.ImageCollection('MODIS/006/MOD44W').filterDate('2015-01-01', '2015-05-01')
    # seleccionar la banda de calidad 
    qa = mod44w.select(['water_mask_QA']).mosaic()
    # bit 4 es la máscara de océano para water_mask_QA
    ocean_bitmask = 4

    return image.updateMask(qa.eq(ocean_bitmask))

In [43]:
collection = collection.map(subset).map(ocean_mod44w_mask).map(square_angle_correction)

## **El efecto sal pimienta y el filtro refinado de Lee (RLF)**
El efecto sal y pimienta es el responsable de dar el aspecto granular de variación espacial aleatoria a las imágenes de SAR, como consecuencia de los principios de medición, y es considerado un ruido complejo. Actualmente, existen diversas técnicas de filtración que tienen como propósito eliminar este ruido, uno de estos es el Filtro Refinado de Lee (RLF) propuesto por [Yommy, Liu y Wu (2015)](https://ieeexplore.ieee.org/document/7334965). Estos autores introducen una mejora usando el algoritmo de K-Nearest Neighbour, logrando reducir el ruido mientras aseguran al mismo tiempo que los bordes, texturas y otras características de la imagen SAR se conserven.  

In [44]:
def refined_lee(image):
    """Método de despeckling conocido como Filtro Refinado de Lee (RLF), basado en el método de Lee y el algoritmo de K-Nearest Neighbour (KNN). 
        Esta es una adaptación para la API de Python del algoritmo desarrollado por Guido Lemoine."""

    # establecer los pesos
    weight = ee.List.repeat(ee.List.repeat(1, 3), 3)

    # crear un kernel 3x3 : width, height, weights, x, y, normalize
    kernel = ee.Kernel.fixed(3, 3, weight, 1, 1, normalize=False)

    # calcular la media y variancia en el kernel 3x3
    mean = image.reduceNeighborhood(ee.Reducer.mean(), kernel)
    var = image.reduceNeighborhood(ee.Reducer.variance(), kernel)

    # determinar los gradientes y las direcciones usando una ventana de 3x3 en el kernel 7x7
    sample_weight = ee.List([[0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 1, 0],
                             [0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 1, 0],
                             [0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 1, 0],
                             [0, 0, 0, 0, 0, 0, 0]])

    # muestra de 3x3 en el kernel 7x7
    sample_kernel = ee.Kernel.fixed(7, 7, sample_weight, 3, 3, normalize=False)

    # calcular la media y variancia en el kernel 7x7 y convertir los vecinos cercanos en múltiples bandas
    sample_mean = mean.neighborhoodToBands(sample_kernel)
    sample_var = var.neighborhoodToBands(sample_kernel)

    # determinar 4 gradientes para la ventana de sampleo
    gradients = sample_mean.select(1).subtract(sample_mean.select(7).abs())
    gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs())
    gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs())
    gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())

    # encontrar el gradiente máximo a lo largo del gradiente de las bandas
    max_gradient = gradients.reduce(ee.Reducer.max())

    # crear una mascara para las bandas donde los píxeles sean el máximo gradiente
    gradient_mask = gradients.eq(max_gradient)

    # duplicamos para representar 2 direcciones
    gradient_mask = gradient_mask.addBands(gradient_mask)

    # determinar las 8 direcciones
    directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(7))).multiply(1)
    directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2))
    directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3))
    directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4))

    # las siguiente son not() de las 4 previas
    directions = directions.addBands(directions.select(0).Not().multiply(5))
    directions = directions.addBands(directions.select(1).Not().multiply(6))
    directions = directions.addBands(directions.select(2).Not().multiply(7))
    directions = directions.addBands(directions.select(3).Not().multiply(8))

    # enmascarar los valores que no son 1-8
    directions = directions.updateMask(gradient_mask)

    # colapsar el stack a una banda
    directions = directions.reduce(ee.Reducer.sum())

    # calcular las estadísticas locales
    sample_stats = sample_var.divide(sample_mean.multiply(sample_mean))

    # calcular la variancia local del ruido
    sigma_v = sample_stats.toArray().arraySort().arraySlice(0, 0, 5).arrayReduce(ee.Reducer.mean(), [0])

    # configurar el kernel 7x7 para todas las direcciones
    rect_weight = ee.List.repeat(ee.List.repeat(0, 7), 3).cat(ee.List.repeat(ee.List.repeat(1, 7), 4))

    diag_weight = ee.List([[1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0],
                           [1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0],
                           [1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 0],
                           [1, 1, 1, 1, 1, 1, 1]])

    rect_kernel = ee.Kernel.fixed(7, 7, rect_weight, 3, 3, False)
    diag_kernel = ee.Kernel.fixed(7, 7, diag_weight, 3, 3, False)

    # crear un stack para la media y variancia usando el kernel original
    dir_mean = image.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1))
    dir_var = image.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1))

    dir_mean = dir_mean.addBands(image.reduceNeighborhood(ee.Reducer.mean(), diag_kernel).updateMask(directions.eq(2)))
    dir_var = dir_var.addBands(image.reduceNeighborhood(ee.Reducer.variance(), diag_kernel).updateMask(directions.eq(2)))

    # añadir bandas y rotar los kernels
    for i in range(1, 4):
        dir_mean = dir_mean.addBands(image.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)).updateMask(directions.eq(2 * i + 1)))
        dir_var = dir_var.addBands(image.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)).updateMask(directions.eq(2 * i + 1)))
        dir_mean = dir_mean.addBands(image.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)).updateMask(directions.eq(2 * i + 2)))
        dir_var = dir_var.addBands(image.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)).updateMask(directions.eq(2 * i + 2)))

    # colapsar el stack en una sola banda
    dir_mean = dir_mean.reduce(ee.Reducer.sum())
    dir_var = dir_var.reduce(ee.Reducer.sum())

    # generamos los valores filtrados
    var_x = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigma_v)).divide(sigma_v.add(1.0))
    b = var_x.divide(dir_var)
    result = dir_mean.add(b.multiply(image.subtract(dir_mean)))

    return result.arrayFlatten([['sum']])
    # return result

Hay que considerar que el filtro RLF opera sobre unidades de sigma naught ($\sigma°$), por lo que se debe **convertir las unidades de SAR** de decibeles (dB) a $\sigma°$ para computar el filtro.


In [45]:
def to_sigma0(image):
    """Convertir unidades en decibeles a sigma naught"""
    return ee.Image(10.0).pow(image.divide(ee.Number(10.0)))

def to_db(image):
    """Convertir unidades en sigma naught a decibeles"""
    return image.log10().multiply(ee.Number(10))

In [46]:
collection = collection.select('VV').map(to_sigma0).map(refined_lee).map(to_db).map(lambda img: img.rename('VV'))

## **Colección de texturas**
La Matriz de Co-Ocurrencia de Niveles Grises (GLCM) es una técnica que permite incrementar el número de dimensiones de nuestro de conjunto de datos usando las propiedades texturales de los objetos y la dependencia espacial de los niveles de grises. Esta implementación fue propuesta por [Haralick (1973)](http://doi.org/10.1109/TSMC.1973.4309314) y [Conners, et al (1984)](http://doi.org/10.1016/0734-189X(84)90197-X).


In [47]:
def texture_metrics(image):
    """Devuelve métricas de textura a partir de la Matriz de Gray Level Co-occurrence Matrix para cada banda."""
    kernel = ee.Kernel.circle(1)
    return image.unitScale(0, 1).int32().glcmTexture(3, kernel)

In [48]:
texture = collection.map(texture_metrics)
collection = collection.combine(texture)

In [49]:
def lowpass_filter(image):
    kernel = ee.Kernel.square(7, 'pixels', True)
    return image.convolve(kernel)

In [50]:
collection = collection.map(lowpass_filter)

## **Segmentación basada en clusterización**

In [51]:
def features_to_numpy(feature_collection, bands):
    size = len(bands)
    input_dict = feature_collection.reduceColumns(ee.Reducer.toList().repeat(size), bands)
    input_list = ee.List(input_dict.get('list')).getInfo()
    return np.array(input_list).T

In [52]:
def get_bandname(collection):
    band_id = collection.toBands().bandNames().getInfo()
    slide = lambda band_id: band_id[68:]
    band_names = [slide(i) for i in band_id]
    
    return band_names

### **Preparación de datos**

In [53]:
# preparar el input
bandname = get_bandname(collection)
training_bands = ['VV', 'VV_asm', 'VV_contrast', 'VV_var', 'VV_ent', 'VV_idm', 'VV_diss', 'VV_inertia']
input = collection.toBands().rename(bandname).select(training_bands)
training = input.sample(region= aoi, scale= 10, numPixels= 1e5)

In [54]:
df = features_to_numpy(feature_collection = training, bands = training_bands)
df = pd.DataFrame(data= df, columns=training_bands) 
display(df.head(5), df.shape)

,VV,VV_asm,VV_contrast,VV_var,VV_ent,VV_idm,VV_diss,VV_inertia
0,-13.101998,0.157851,0.870106,1.114735,2.263351,0.712013,0.624286,0.870106
1,-12.590091,0.249400,0.580053,0.560155,1.875519,0.771751,0.477090,0.580053
2,-12.929841,0.146724,0.850423,1.066222,2.271129,0.723744,0.601640,0.850423
3,-12.739207,0.144670,0.827037,1.045271,2.314578,0.717522,0.608413,0.827037
4,-12.949391,0.222920,0.655238,0.762306,1.947986,0.768254,0.495450,0.655238


(93965, 8)

## **Prueba de tendencia de agrupamiento**

Los algoritmos de agrupamiento tienden a generar clusters incluso cuando son aplicados sobre datos aleatorios [Dubes y Jain (1979)](https://doi.org/10.1016/0031-3203(79)90034-7). Es por esto, que la validación de la tendencia de agrupamiento natural en un conjunto de datos resulta en un paso de vital importancia. La prueba estadística utilizada es la de Hopkins, propuesta por [Banerjee y Dave (2004)](https://doi.org/10.1109/FUZZY.2004.1375706)



In [55]:
def hopkins(X):
    from sklearn.neighbors import NearestNeighbors
    from random import sample
    
    n, d = X.shape
    m = int(0.1 * n) # heuristic

    # Cada punto xi ∈ D, encuentra su vecino cercano xj
    nbrs = NearestNeighbors(n_neighbors=1).fit(X.values)
    rand = np.random.choice(n, m)
  
    uj = []
    wj = []
    for j in range(0, m):
      u_dist, _ = nbrs.kneighbors(np.random.uniform(np.amin(X, axis=0), np.amax(X, axis=0), d).reshape(1, -1), 2, return_distance=True)
      uj.append(u_dist[0][1])
      w_dist, _ = nbrs.kneighbors(X.iloc[rand[j]].values.reshape(1, -1), 2, return_distance=True)
      wj.append(w_dist[0][1])
    
    H = np.sum(uj) / (np.sum(uj) + np.sum(wj))
  
    return H

In [56]:
hopkins(df)

0.9919238830251714

## **Modelo de Clustering**

**Algoritmo Learning Vector Quantization (LVQ)**

**Parámetros del modelo**
> - **numClusters** : El número de clusters debe ser mayor a 0. Un cluster es una solución trivial, por lo que el mínimo deben ser 2. 
> - **learningRate** : El ratio de aprendizaje debe ser mayor a 0.01 y menor a 1. 
> - **epochs** : El número de épocas debe ser mayor a 1. Se establece por defecto 1000 épocas.


In [57]:
clusterer = ee.Clusterer.wekaLVQ(numClusters=3, 
                                 learningRate=0.01, 
                                 epochs=1e3, 
                                 normalizeInput=True)\
                                 .train(training)

In [58]:
print('Variables de entrada de LVQ: {}'.format(clusterer.schema().getInfo()))

Variables de entrada de LVQ: ['VV', 'VV_asm', 'VV_contrast', 'VV_var', 'VV_ent', 'VV_idm', 'VV_diss', 'VV_inertia']


### **Implementación**

In [59]:
clusters = input.cluster(clusterer)

### **Vectorización**

Google Earth Engine opera de forma piramidal, esta premisa es importante, puesto que las salidas del algoritmo de clustering se dan en teselas, que varían en función de la escala de visualización, resultando conveniente su transformación a vector, estableciendo la escala deseada.

In [65]:
cluster_vector = clusters.eq(1).selfMask().reduceToVectors(scale=10)

## **Mapa interactivo**

In [66]:
# Configurar el token y atribución de Mapbox
basemap = 'https://api.mapbox.com/styles/v1/juancn95/ck8rm02y905bc1jmkwjbfd4ag/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoianVhbmNuOTUiLCJhIjoiY2s4NTBrZ3FuMDF5bzNpcDF6MnN3Y3h3MCJ9.GM4l-z4U70YMLHd2KCVitQ'
attribution = '© <a href="https://www.mapbox.com/about/maps/">Mapbox</a> © <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>'

# Crear un mapa
map = folium.Map(location=[lat, lon], 
                 height='100%',
                 width='100%',
                 tiles= basemap,
                 attr= attribution,
                 zoom_start=10,
                 prefer_canvas=True)

# Crear un TileLayer para el minimap
basemap_mini = folium.raster_layers.TileLayer(tiles= basemap, 
                                              attr= attribution)
# Crear un minimap
mini_map = MiniMap(tile_layer = basemap_mini,
                   collapsed_width=50,
                   zoom_level_offset=-6,
                   width=300, 
                   height=100)
# Añadir el minimapa
map.add_child(mini_map)

# Añadimos el vector de Google Earth Engine
cluster_vector = clusters.eq(1).selfMask().reduceToVectors(scale=10)
draw = cluster_vector.draw(color="red", strokeWidth=1).getMapId()
fet = draw["tile_fetcher"].url_format
folium.raster_layers.TileLayer(tiles=fet,
                                attr='Google Earth Engine',
                                name='Cluster').add_to(map)

map

# **Referencias Bibliográficas**

- Gorelick, N., Hancher, M., Dixon, M., Ilyushchenko, S., Thau, D., & Moore, R. (2017). Google Earth Engine: Planetary-scale geospatial analysis for everyone. Remote sensing of Environment, 202, 18-27.
- Alpers, W., & Huang, W. (2010). On the discrimination of radar signatures of atmospheric gravity waves and oceanic internal waves on synthetic aperture radar images of the sea surface. IEEE Transactions on Geoscience and Remote Sensing, 49(3), 1114-1126.
- Singha, S., Vespe, M., & Trieschmann, O. (2013). Automatic Synthetic Aperture Radar based oil spill detection and performance estimation via a semi-automatic operational service benchmark. Marine pollution bulletin, 73(1), 199-209.
- Yommy, A. S., Liu, R., & Wu, S. (2015). SAR image despeckling using refined Lee filter. In 2015 7th International Conference on Intelligent Human-Machine Systems and Cybernetics (Vol. 2, pp. 260-265). IEEE.
- Robert, H., Shanmugan, K., & Dinstein, I. (1973). Textural features for image classification.
- Conners, R. W., Trivedi, M. M., & Harlow, C. A. (1984). Segmentation of a high-resolution urban scene using texture operators. Computer vision, graphics, and image processing, 25(3), 273-310.
- Dubes, R., & Jain, A. K. (1979). Validity studies in clustering methodologies. Pattern recognition, 11(4), 235-254.
- Banerjee, A., & Dave, R. N. (2004). Validating clusters using the Hopkins statistic. In 2004 IEEE International conference on fuzzy systems (IEEE Cat. No. 04CH37542) (Vol. 1, pp. 149-153). IEEE.
- T. Kohonen. (2003). "Learning Vector Quantization". The Handbook of Brain Theory and Neural Networks. MIT Press, 631-634.
